In [ ]:
# Importing required packages

import pandas as pd
import numpy as np

import os
import gc

from ta_lib.core.api import load_yml, create_db_connection, create_context, _change_permissions_recursive

import datetime

In [ ]:
# Specifying config and connections
config_path = '../conf/config.yml'
cfg = create_context(config_path).offline

conn=create_db_connection(cfg)

In [ ]:
def get_missing_data(start_date, end_date) :
    
    sql_query= f"""
    SELECT 
    pos_trn_id, pos_trn_itm_seq_nu, pos_ord_nu,
    pos_ord_dt,  pos_itm_tot_qt, pos_itm_net_unt_prc_am,  pos_itm_grss_unt_prc_am,
    pos_tot_disc_am,  pos_tot_disc_pc, pos_itm_unt_tot_cost_am,  pos_itm_grss_am,
    pos_itm_tot_cost_am, mcd_gbal_lcat_id_nu, cmbo_pren_sld_menu_itm_id,
    pos_busn_dt, pos_ord_beg_tm, dypt_id_nu, terr_pos_trn_typ_cd,
    terr_pymt_meth_cd, terr_prd_dlvr_meth_cd,  terr_pos_area_cd, sld_menu_itm_id
    FROM RMDW_Tables.POS_TRN_ITM
    WHERE terr_pos_trn_typ_cd in ('105')
    AND POS_ITM_GVAW_ITM_FL = 0
    AND CAST(POS_BUSN_DT as DATE) >= CAST('{start_date}' as DATE)
    AND CAST(POS_BUSN_DT as DATE) <= CAST('{end_date}' as DATE)
    """    
    return sql_query

In [ ]:
# Defining variables
start_date = '2021-08-01'
end_date = '2021-08-31'

date_list = [x.strftime('%Y-%m-%d') for x in pd.date_range(start_date,end_date,freq='w').to_list()] + ['2021-08-31']

date_list

In [ ]:
op_path = '/opt/sasdata/dev/PricingEngines/DiscountEngines/Russia/data/raw/redshift_105_trn_1/'

stat_df = pd.DataFrame()
for i in range(len(date_list)-1) :
    
    t1 = pd.Timestamp.today()
    
    if i == len(date_list)-2 :
        start_date = date_list[i]
        end_date = date_list[i+1]
    else :   
        start_date = date_list[i]
        end_date = (pd.to_datetime(date_list[i+1]) - datetime.timedelta(1)).date()
        
    query_q = get_missing_data(start_date, end_date)
    
    df = pd.read_sql(query_q, conn)
    
    os.makedirs(op_path, exist_ok=True)
    df.to_parquet(op_path + start_date.replace('-','') + '.parquet')
    _change_permissions_recursive(op_path, 0o777)
    
    t2 = pd.Timestamp.today()
    
    temp_df = pd.DataFrame()
    temp_df['start_date'] = [ start_date ]
    temp_df['end_date'] = [ end_date ]
    temp_df['rows'] = [ df.shape[0] ]
    temp_df['cols'] = [ df.shape[1] ]
    temp_df['time taken'] = [ t2-t1 ]
    
    stat_df = pd.concat([stat_df, temp_df], axis=0)
    
    print(date_list[i], 'is done in ', t2-t1)
    


In [ ]:
stat_df